In [17]:
import sys
sys.path.append("E:\College Stuff\Semester 7\Final Year Project\Review 3\Resemblyzer")
import os
import math
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from spectralcluster import SpectralClusterer, constraint, RefinementOptions, ThresholdType, ICASSP2018_REFINEMENT_SEQUENCE, AutoTune, LaplacianType
from pydub import AudioSegment
import librosa
import numpy as np
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [18]:
# give the file path to your audio file
audio_file_path = "E:\College Stuff\Semester 7\Final Year Project\Review 3\sample4.wav"
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

audio = AudioSegment.from_file(wav_fpath)
audio_length = audio.duration_seconds

# refinement_options = RefinementOptions(
#     gaussian_blur_sigma=1,
#     p_percentile=0.95,
#     thresholding_soft_multiplier=0.01,
#     thresholding_type=ThresholdType.RowMax,
#     refinement_sequence=ICASSP2018_REFINEMENT_SEQUENCE)

# autotune = AutoTune(
#     p_percentile_min=0.60,
#     p_percentile_max=0.95,
#     init_search_step=0.01,
#     search_level=3)

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=7)
#     autotune=autotune,
#     laplacian_type=LaplacianType.RandomWalk,
#     refinement_options=refinement_options,
#     custom_dist="cosine")

labels = clusterer.predict(cont_embeds)

Loaded the voice encoder model on cpu in 1.08 seconds.
(1840, 256)


In [20]:
def create_labelling(labels, wav_splits):
    from resemblyzer import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0
    
    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling

In [21]:
labelling = create_labelling(labels,wav_splits)

for i in labelling:
    if i[2] - i[1] < 0.7:
        labelling.remove(i)

labelling.append((str(labelling[-1][0]), labelling[-1][2], math.ceil(audio_length)))
print(labelling)

[('1', 0, 7.7), ('0', 7.7, 8.48), ('1', 8.48, 14.24), ('0', 14.24, 17.12), ('1', 17.12, 28.52), ('0', 28.52, 45.68), ('0', 45.74, 45.86), ('1', 45.86, 48.56), ('0', 48.56, 74.3), ('1', 74.3, 83.54), ('0', 83.54, 101.24), ('1', 101.24, 107.3), ('0', 107.3, 111.14), ('0', 111.14, 120)]


In [22]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
def transcribe(wav_fpath, start, duration):
    input_audio, sample_rate = librosa.load(wav_fpath, sr=16000, offset = start ,duration = duration)
    inputs = processor(input_audio, sampling_rate=sample_rate, return_tensors="pt").input_values
    logits = model(inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    return processor.batch_decode(predicted_ids)

In [24]:
Transcription_101 = []
for i in labelling:
    start, end = math.floor(i[1]),  math.ceil(i[2])
    while ((end - start) >= 8): 
        transcription = transcribe(wav_fpath, start, 8)
        Transcription_101.append([i[0], transcription[0]])
        start += 8
    if start == end:
        continue
    transcription = transcribe(wav_fpath, start, int(end - start))
    Transcription_101.append([i[0], transcription[0]])
Transcription_101

[['1',
  'SO YOU ARE HERE WITH YOUR DAUGHTER WHO IS TWENTY ONE RIGHT AND YOU WERE TWENTY ONE WHEN YOU BECAME A MILLIONAIRE IS THAT RIGHT'],
 ['0', 'IS THAT RIGHT ALMOST'],
 ['1',
  'ALMOST ALL RIGHT SOAROUND THAT AGE OU WO YOU WERE LIKE THE YOUNGEST PERSON TO BECOME A MILLIONAIRE IS THAT RIGHT'],
 ['0', 'TATRIGHT PRETI GOT IN TERMS OF MY OWN EARNING IT ON MY'],
 ['1',
  'EARNING IT ON MY OWN YOW RIGHT O K SO WELL JITS THE MOST IMPORTANT THINGS E SO DID YOU WHEN YOU WERE A KID DID YOU'],
 ['1',
  "DID YOU CARE ABOUT MONEY OR YOU JUST CARED ABOUT TECHNOLOGY AND THAT'S IT JUST"],
 ['0',
  "THAT'S IT JUST HAPPENED MOSTLY I LOVE SOFT WEAR I DO REMEMBER AT THE PRIVATE SCHOOL I WENT TO THER OTHER KIDS WHOSE FAMILIES"],
 ['0',
  "WERE BETTER OFF LIKE THEY HAD A PORCH OR SOMETHING BUT IT WASN'T THAT THAT BIG OF A DEAL MY THING WAS THAT I JUST LOVED DOING SOFT WHERE I LOVED"],
 ['0', 'TIRING PEOPLE AND I WAS'],
 ['0', 'AND I WAS'],
 ['1', 'AND I WAS STUNNED WHEN IT ENDED UP BEING SO VALUABLE'],


In [25]:
Transcription = []
i = 0
while i < (len(Transcription_101) - 1):
    trans = Transcription_101[i][1]
    while (i < (len(Transcription_101) - 1)) and (Transcription_101[i+1][0] == Transcription_101[i][0]):
        trans += " " + Transcription_101[i+1][1]
        i += 1
    Transcription.append(f"Speaker {Transcription_101[i][0]} --> {trans}\n")
    i += 1
Transcription

['Speaker 1 --> SO YOU ARE HERE WITH YOUR DAUGHTER WHO IS TWENTY ONE RIGHT AND YOU WERE TWENTY ONE WHEN YOU BECAME A MILLIONAIRE IS THAT RIGHT\n',
 'Speaker 0 --> IS THAT RIGHT ALMOST\n',
 'Speaker 1 --> ALMOST ALL RIGHT SOAROUND THAT AGE OU WO YOU WERE LIKE THE YOUNGEST PERSON TO BECOME A MILLIONAIRE IS THAT RIGHT\n',
 'Speaker 0 --> TATRIGHT PRETI GOT IN TERMS OF MY OWN EARNING IT ON MY\n',
 "Speaker 1 --> EARNING IT ON MY OWN YOW RIGHT O K SO WELL JITS THE MOST IMPORTANT THINGS E SO DID YOU WHEN YOU WERE A KID DID YOU DID YOU CARE ABOUT MONEY OR YOU JUST CARED ABOUT TECHNOLOGY AND THAT'S IT JUST\n",
 "Speaker 0 --> THAT'S IT JUST HAPPENED MOSTLY I LOVE SOFT WEAR I DO REMEMBER AT THE PRIVATE SCHOOL I WENT TO THER OTHER KIDS WHOSE FAMILIES WERE BETTER OFF LIKE THEY HAD A PORCH OR SOMETHING BUT IT WASN'T THAT THAT BIG OF A DEAL MY THING WAS THAT I JUST LOVED DOING SOFT WHERE I LOVED TIRING PEOPLE AND I WAS AND I WAS\n",
 'Speaker 1 --> AND I WAS STUNNED WHEN IT ENDED UP BEING SO VALUAB

In [26]:
path = "E:/College Stuff/Semester 7/Final Year Project/Review 3/transcription.txt"
file = open(path,"w+")
file.writelines(Transcription)
file.close()

In [42]:
# from punctuator import Punctuator
# p = Punctuator('model.pcl')
# print(p.punctuate(transcription))
# print(p.punctuate(parser.parse(transcription)['result']))